# Branch & Bound and Knapsack Lab

**Objectives**
- Preform the branch and bound algorithm
- Apply branch and bound to the knapsack problem
- Understand the geometry of the branch and bound algorithm

**Brief description:** In this lab, we will try solving an example of a knapsack problem with the branch-and-bound algorithm. We will also see how adding a cutting plane helps in reducing the computation time and effort of the algorithm. Lastly, we will explore the geometry of the branch and bound algorithm.

In [ ]:
# imports -- don't forget to run this cell
import pandas as pd
import gilp
from gilp.visualize import feasible_integer_pts
from ortools.linear_solver import pywraplp as OR

## Part 1: Branch and Bound Algorithm

Recall that the branch and bound algorithm (in addition to the simplex method) allows us to solve integer programs. Before applying the branch and bound algorithm to the knapsack problem, we will begin by reviewing some core ideas. Furthermore, we will identify a helpful property that will make branch and bound terminate quicker later in the lab!

**Q1:** What are the different ways a node can be fathomed during the branch and bound algorithm? Describe each.

**A:** Either a feasible solution is not found, or the feasible solution's value is greater than or equal to the best integer solution already found.

**Q2:** Suppose you have a maximization integer program and you solve its linear program relaxation. What does the LP-relaxation optimal value tell you about the IP optimal value? What if it is a minimization problem?

**A:** For a maximization problem, the linear program relaxation's solution gives an upper bound to the IP optimal value. For a minimization problem, the LP-relaxation's solution gives a lower bound to the IP optimal value.

**Q3:** Assume you have a maximization integer program with all integral coefficents in the objective function. Now, suppose you are running the branch and bound algorithm and come across a node with an optimal value of 44.5. The current incumbent is 44. Can you fathom this node? Why or why not?

**A:** Yes. This is because the current best IP solution gives a value of 44, meaning the bound to any feasible solution is at most 44. Therefore, we can fathom this node because 44.5 is greater than 44.

**Q4:** If the optimal solution to the LP relaxation of the original program is integer, then you have found an optimal solution to your integer program. Explain why this is true.

**A:** The LP-relaxation and the IP place opposite bounds on each other, meaning that the solutions will be optimal in both cases, creating an optimal solution.

**Q5:** If the LP is infeasible, then the IP is infeasible. Explain why this is true.

**A:** Since the LP places some bound on the IP solution, if there is no feasible solution to the LP, then there cannot be an optimal solution to the IP.

The next questions ask about the following branch and bound tree. If the solution was not integral, the fractional $x_i$ that was used to branch is given. If the solution was integral, it is denoted *INT*. In the current iteration of branch and bound, you are looking at the node with the *****.

<img src="part1_bnb_tree.png" width="500"/>

**Q6:** Can you determine if the integer program this branch and bound tree is for is a minimization or maximixation problem? If so, which is it?

**A:** This problem is a minimization problem since the original value is the lowest of the other values found in the tree.

Hint: For **Q7-8**, you can assume integral coefficents in the objective function.

**Q7:** Is the current node (marked $z^*$) fathomed? Why or why not? If not, what additional constraints should be imposed for each of the next two nodes?

**A:** No, since the next best optimal integer value is 17, which is less than the only other integer solution found. The constraints that should be imposed would be either x1 <= 1 or x1 >= 0.

**Q8:** Consider the nodes under the current node (where $z = 16.3$). What do you know about the optimal value of these nodes? Why? 

**A:** The optimal value of these nodes is at least 16.3. Subsequently, since the coefficients are integral, the optimal solution is limited further to at least 17.

## Part 2: The Knapsack Problem

In this lab, you will solve an integer program by branch and bound. The integer program to be solved will be a knapsack problem.

**Knapsack Problem:** We are given a collection of $n$ items, where each item $i = 1,\dots,n$ has a weight $w_i$ and a value $v_i$. In addition, there is a given capacity $W$, and the aim is to select a maximum value subset of items that has a total weight at most $W$. Note that each item can be brought at most once.

$$\begin{align*}
\max \quad & \sum_{i=1}^n v_ix_i\\
\text{s.t.} \quad & \sum_{i=1}^n w_ix_i \leq W \\
& 0 \leq x_i \leq 1, \text{integer}, i = 1,\dots,n
\end{align*}$$

Consider the following data which we import from a CSV file:

In [ ]:
data = pd.read_csv('knapsack_data_1.csv', index_col=0)
data

and $W = 18$. 

**Q9:** Are there any items we can remove from our input to simplify this problem? Why? If so, replace `index` with the item number that can be removed in the code below. Hint: how many of each item could we possibly take?

**A:** We can remove item 7 since its weight alone exceeds the capacity constraint that has been given in the problem.

In [ ]:
# TODO: replace index
data = data.drop(7)

**Q10:** If we remove item 7 from the knapsack, it does not change the optimal solution to the integer program. Explain why.

**A:** This is because item 7 could never have been included in any feasible solution since its weight alone exceeds the weight constraint.

**Q11:** Consider removing items $i$ such that $w_i > W$ from a knapsack input. How does the LP relaxation's optimal value change?

**A:** Since the LP relaxation would allow for partial items to be taken, it would then limit these possibilities for an optimal value.

In **Q10-11**, you should have found that removing these items removes feasible solutions from the linear program  but does not change the integer program. This is desirable as the gap between the optimal IP and LP values can become smaller. By adding this step, branch and bound may terminate sooner. 

Recall that a branch and bound node can be fathomed if its bound is no better than the value of the best feasible integer solution found thus far. Hence, it helps to have a good feasible integer solution as quickly as possible (so that we stop needless work). To do this, we can first try to construct a good feasible integer solution by a reasonable heuristic algorithm before starting to run the branch and bound procedure.

In designing a heuristic for the knapsack problem, it is helpful to think about the value per unit weight for each item. We compute this value in the table below.

In [ ]:
data['value per unit weight'] = (data['value'] / data['weight']).round(2)
data

**Q12:** Design a reasonable heuristic for the knapsack problem. Note a heuristic aims to find a decent solution to the problem (but is not necessarily optimal).

**A:** A reasonable heuristic will be to take the most valuable item per unit weight. Then, find the next most valuable item per unit weight that can be taken without exceeding the weight constraint. Continue this process until the weight constraint has been reached.

**Q13:** Run your heuristic on the data above to compute a good feasible integer solution. Your heuristic should generate a feasible solution with a value of 64 or better. If it does not, try a different heuristic (or talk to your TA!)

**A:** Using the heuristic  above would result in taking item 1 and item 4 or 5, yielding a value of 64 or 62. From this, taking item 4 is obviously better than taking item 5, so the heuristic gives the solution that involves taking items 1 and 4 (value 64).

We will now use the branch and bound algorithm to solve this knapsack problem! First, let us define a mathematical model for the linear relaxation of the knapsack problem.

**Q14:** Complete the model below.

In [ ]:
def Knapsack(table, capacity, integer = False):
    """Model for solving the Knapsack problem.
    
    Args:
        table (pd.DataFrame): A table indexd by items with a column for value and weight
        capcity (int): An integer-capacity for the knapsack
        integer (bool): True if the variables should be integer. False otherwise.
    """
    ITEMS = list(table.index)        # set of items
    v = table.to_dict()['value']     # value for each item 
    w = table.to_dict()['weight']    # weight for each item
    W = capacity                     # capacity of the knapsack
    
    # define model
    m = OR.Solver('knapsack', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i in ITEMS:
        if integer:
            x[i] = m.IntVar(0, 1, 'x_%d' % (i)) 
        else:
            x[i] = m.NumVar(0, 1, 'x_%d' % (i)) 
        
    # define objective function here
    m.Maximize(sum(v[i]*x[i] for i in ITEMS))
    
    # TODO: Add a constraint that enforces that weight must not exceed capacity
    # recall that we add constraints to the model using m.Add()
    m.Add(sum(w[i]*x[i] for i in ITEMS) <= W)
    
    return (m, x)  # return the model and the decision variables

In [ ]:
# You do not need to do anything with this cell but make sure you run it!
def solve(m):
    """Used to solve a model m."""
    m.Solve()
    
    print('Objective =', m.Objective().Value())
    print('iterations :', m.iterations())
    print('branch-and-bound nodes :',m.nodes())
    
    return ({var.name() : var.solution_value() for var in m.variables()})

We can now create a linear relaxation of our knapsack problem. Now, `m` represents our model and `x` represents our decision variables.

In [ ]:
m, x = Knapsack(data, 18)

We can use the next line to solve the model and output the solution

In [ ]:
solve(m)

**Q15:** How does this optimal value compare to the value you found using the heuristic integer solution?

**A:** This value is higher.

**Q16:** Should this node be fathomed? If not, what variable should be branched on and what additional constraints should be imposed for each of the next two nodes?


**A:** This should not be fathomed yet since we are attempting to maximize the objective value. Since this value is greater than the one found by the heuristic, we should branch on the non-integer decision variable, x2. Therefore, the constraints x2 >= 1 and x2 <= 0

After constructing the linear relaxation model using `Knapsack(data1, 18)` we can add additional constraints. For example, we can add the constraint $x_2 \leq 0$ and solve it as follows:

In [ ]:
m, x = Knapsack(data, 18)
m.Add(x[2] <= 0)
solve(m)

 **NOTE:** The line `m, x = Knapsack(data1, 18)` resets the model `m` to the LP relaxation. All constraints from branching have to be added each time.

**Q17:** Use the following cell to compute the optimal value for the other node you found in **Q16**.

In [ ]:
# TODO: Answer Q17
m, x = Knapsack(data, 18)
m.Add(x[2] >= 1)
solve(m)

**Q18:** What was the optimal value? Can this node be fathomed? Why? (Hint: In **Q13**, you found a feasible integer solution with value 64.)

**A:** So far, the optimal value is at most 69, since the value found when x2 is constrained to x2 <= 0 was 69.2. Therefore, since we have found a bound greater than 64 (as found in the heuristic) but not an integer solution, we cannot fathom this node yet.

If we continue running the branch and bound algorithm, we will eventually reach the branch and bound tree below where the $z^*$ indictes the current node we are looking at.

<img src="part2_bnb_tree.png" width="700"/>

**Q19:** The node with $z = 64.857$ was fathomed. Why are we allowed to fathom this node? (Hint: think back to **Q3**)

**A:** Since the bound imposed by this solution is the next integer solution below 64.857 (which is 64), it can be said that there is no better solution by imposing this constraint. Therefore, we can fathom this node.

**Q20:** Finish running branch and bound to find the optimal integer solution. Use a separate cell for each node you solve and indicate if the node was fathomed with a comment. (Hint: Don't forget to include the constraints further up in the branch and bound tree.)

In [ ]:
# Template
m, x = Knapsack(data, 18)
# Add constraints here
m.Add(x[2] <= 0)
m.Add(x[3] <= 0)
m.Add(x[4] >= 1)
m.Add(x[5] <= 0)
m.Add(x[6] >= 1)
solve(m)
# fathomed? y

In [ ]:
# Template
m, x = Knapsack(data, 18)
# Add constraints here
m.Add(x[2] <= 0)
m.Add(x[3] <= 0)
m.Add(x[4] >= 1)
m.Add(x[5] <= 0)
m.Add(x[6] <= 0)
solve(m)
# fathomed? y

**A:** The optimal solution was found by taking x1 and x4 as found by the original heuristic.

**Q21:** How many nodes did you have to explore while running the branch and bound algorithm?

**A:** Five different nodes had to be explored in order to find a verified integer solution.

In the next section, we will think about additional constraints we can add to make running branch and bound quicker.

## Part 3: Cutting Planes

In general, a cutting plane is an additional constraint we can add to an integer program's linear relaxation that removes feasible linear solutions but does not remove any integer feasible solutions. This is very useful when  solving integer programs! Recall many of the problems we have learned in class have something we call the "integrality property". This is useful because it allows us to ignore the integrality constraint since we are garunteed to reach an integral solution. By cleverly adding cutting planes, we strive to remove feasible linear solutions (without removing any integer feasible solutions) such that the optimal solution to the linear relaxation is integral!

Conisder an integer program whose linear program releaxation is 

$$\begin{align*}
\max \quad & 2x_1+x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 3 \\
& 2x_1 \leq 5 \\
& -x_1 + 2x_2 \leq 2 \\
& x_1, x_2 \geq 0
\end{align*}$$

We can define this linear program and then visualize its feasible region. The integer points have been highlighted.

In [ ]:
lp = gilp.LP([[1,1],[2,0],[-1,2]],
             [3,5,2],
             [2,1])
fig = gilp.lp_visual(lp)
fig.set_axis_limits([3.5,2])
fig.add_trace(feasible_integer_pts(lp, fig))
fig

**Q22:** List every feasible solution to the integer program.

**A:** (0,0), (0,1), (1,0), (1,1), (2,0), (2,1)

**Q23:** Is the constraint $x_2 \leq 1$ a cutting plane? Why? (Hint: Would any feasible integer points become infeasible? What about feasible linear points?)

**A:** Yes, since the constraint eliminates any linear solutions above x2 = 1 but still allows for x2 to equal 1, preserving the integer solutions.

Let's add this cutting plane to the LP relaxation!

In [ ]:
lp = gilp.LP([[1,1],[2,0],[-1,2],[0,1]],
             [3,5,2,1],
             [2,1])
fig = gilp.lp_visual(lp)
fig.set_axis_limits([3.5,2])
fig.add_trace(feasible_integer_pts(lp, fig))
fig

**Q24:** Is the constraint $x_1 \leq 3$ a cutting plane? Why?

**A:** No since this constraint does not eliminate any feasible solutions.

**Q25:** Can you provide another cutting plane? If so, what is it?

**A:** Instead of x1 <= 3, a better cutting plane would be x1 <= 2.

Let's look at the feasible region after adding the cutting plane from **Q23** and one of the possible answers from **Q25**. Notice the optimal solution to the LP relaxation is now integral!

In [ ]:
lp = gilp.LP([[1,1],[2,0],[-1,2],[0,1],[1,0]],
             [3,5,2,1,2],
             [2,1])
fig = gilp.lp_visual(lp)
fig.set_axis_limits([3.5,2])
fig.add_trace(feasible_integer_pts(lp, fig))
fig

Let's try applying what we know about cutting planes to the knapsack problem! Again, recall our input was $W = 18$ and:

In [ ]:
data

**Q26:** Look at items 1, 2, and 3. How many of these items can we take simultaneously? Can you write a new constraint to capture this? If so, please provide it.

**A:** Of items 1, 2, and 3, we can only take one item since any combination of items will exceed the weight constraint. Therefore, a constraint to reflect this would be x1 + x2 + x3 <= 1

**Q27:** Is the constraint you found in **Q26** a cutting plane? If so, provide a feasible solution to the linear program relaxation that is no longer feasible (i.e. a point the constraint *cuts off*).

**A:** Yes since now it infeasible to take, for example, item 1 and 2/3 item 2.

**Q28:** Provide another cutting plane involving items 4,5 and 6 for this integer program. Explain how you derived it.

**A:** x4 + x5 + x6 <= 2. Similar to the last one, any combination of two items is feasible but not all three.

**Q29:** Add the cutting planes from **Q26** and **Q28** to the model and solve it. You should get a solution in which we take items 1 and 4 and $\frac{1}{6}$ of item 5 with an objective value of 66.

In [ ]:
m, x = Knapsack(data, 18)
# TODO: Add cutting planes here
m.Add(x[1] + x[2] + x[3] <= 1)
m.Add(x[4] + x[5] + x[6] <= 2)

solve(m)

Let's take a moment to pause and reflect on what we are doing. Recall from **Q9-11** that we dropped item 7 becuase its weight was greater than the capcity of the knapsack. Essentially we added the constraint $x_7 \leq 0$. This constraint was a cutting plane! It eliminated some linear feasible solutions but no integer ones. By adding these two new cutting planes, we can get branch and bound to terminate earlier yet again! So far, we have generated cutting planes by inspection. However, there are more algorithmic ways to identify them (which we will ignore for now).

If we continue running the branch and bound algorithm, we will eventually reach the branch and bound tree below where the $z^*$ indictes the current node we are looking at. 

<img src="part3_bnb_tree.png" width="700"/>

**NOTE:** Do not forget about the feasible integer solution our heuristic gave us with value 64.

**Q30** Finish running branch and bound to find the optimal integer solution. Use a separate cell for each node you solve and indicate if the node was fathomed with a comment. Hint: Don't forget the cutting plane constraints should be included in every node of the branch and bound tree.

In [ ]:
# Template
m, x = Knapsack(data, 18)
# Add constraints here
m.Add(x[1] + x[2] + x[3] <= 1)
m.Add(x[4] + x[5] + x[6] <= 2)
m.Add(x[5] >= 1)
m.Add(x[4] >= 1)

solve(m)
# fathomed? y

In [ ]:
# Template
m, x = Knapsack(data, 18)
# Add constraints here
m.Add(x[1] + x[2] + x[3] <= 1)
m.Add(x[4] + x[5] + x[6] <= 2)
m.Add(x[5] >= 1)
m.Add(x[4] <= 0)

solve(m)
# fathomed? y

In [ ]:
# Template
m, x = Knapsack(data, 18)
# Add constraints here
m.Add(x[1] + x[2] + x[3] <= 1)
m.Add(x[4] + x[5] + x[6] <= 2)
m.Add(x[5] >= 1)
m.Add(x[4] <= 0)
m.Add(x[6] >= 1)

solve(m)
# fathomed? y

In [ ]:
# Template
m, x = Knapsack(data, 18)
# Add constraints here
m.Add(x[1] + x[2] + x[3] <= 1)
m.Add(x[4] + x[5] + x[6] <= 2)
m.Add(x[5] >= 1)
m.Add(x[4] <= 0)
m.Add(x[6] <= 0)

solve(m)
# fathomed? y

**A:** Optimal solution is still the solution found by the heuristic and subsequently by the branch and bound with items 1 and 4 being taken with a value of 64.

**Q31:** Did you find the same optimal solution? How many nodes did you explore? How did this compare to the number you explored previously?

**A:** The same optimal solution was found by only exploring 3 nodes instead of 5.

## Part 4: Geometry of Branch and Bound

Previously, we used the `gilp` package to viusualize the simplex algorithm but it also has the functionality to visualize branch and bound. We will give a quick overview of the tool. Similar to `lp_visual` and `simplex_visual`, the function `bnb_visual` takes an `LP` and returns a visualization. It is assumed that every decision variable is constrained to be integer. Unlike previous visualizations, `bnb_visual` returns a series of figures for each node of the branch and bound tree. Let's look at a small 2D example:

$$\begin{align*}
\max \quad & 5x_1+ 8x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 6 \\
& 5x_1 + 9x_2 \leq 45 \\
& x_1, x_2 \geq 0, \quad \text{integral}
\end{align*}$$

In [ ]:
nodes = gilp.bnb_visual(gilp.examples.STANDARD_2D_IP)

In [ ]:
nodes[0].show()

Run the cells above to generate a figure for each node and view the first node. At first, you will see the LP relaxation on the left and the root of the branch and bound tree on the right. The simplex path and isoprofit slider are also present.

**Q32:** Recall the root of a branch and bound tree is the unaltered LP relaxation. What is the optimal solution? (Hint: Use the objective slider and hover over extreme points).

**A:** The optimal solution to the LP relaxation is x1 = 2.25 and x2 = 3.75 with an objective value of 41.25.

**Q33:** Assume that we always choose the variable with the minimum index to branch on if there are multiple options. Write down (in full) each of the LPs we get after branching off the root node.

**A:** 
$$\begin{align*}
\max \quad & 5x_1+ 8x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 6 \\
& 5x_1 + 9x_2 \leq 45 \\
& x_1 \leq 2 \\
& x_1, x_2 \geq 0, \quad \text{integral}
\end{align*}$$

$$\begin{align*}
\max \quad & 5x_1+ 8x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 6 \\
& 5x_1 + 9x_2 \leq 45 \\
& x_1 \geq 3 \\
& x_1, x_2 \geq 0, \quad \text{integral}
\end{align*}$$

**Q34:** Draw the feasible region to each of the LPs from **Q33** on the same picture.

**A:**

Run the following cell to see if the picture you drew in **Q34** was correct.

In [ ]:
nodes[1].show()

The outline of the original LP relaxation is still shown on the left. Now that we have eliminated some of the fractional feasible solutions, we now have 2 feasible regions to consider. The darker one is the feasible region associated with the current node which is also shaded darker in the branch and bound tree. The unexplored nodes in the branch and bound tree are not shaded in.

**Q35:** Which feasible solutions to the LP relaxation are removed by this branch?

**A:** The main one that is removed is the original optimal solution since it was not an integer solution.

**Q36:** At the current (dark) node, what constraints will we add? How many feasible regions will the original LP relaxation be broken into?

**A:** The new constraints will be x2 <= 3 and x2 >= 4, breaking the original region into three smaller ones.

In [ ]:
nodes[2].show()

**Q37:** What is the optimal solution at the current (dark) node? Do we have to further explore this branch? Explain.

**A:** The optimal solution at this node is (2,3) with a value of 34. We could still explore this branch since the bound is currently 41. Therefore, it could be possible that there is a better integer solution. greater than 34 but less than 41.

**Q38:** Recall shaded nodes have been explored and the node shaded darker (and feasible region shaded darker) correspond to the current node and its feasible region. Nodes not shaded have not been explored. How many nodes have not yet been explored?

**A:** 2 nodes have not been explored.

**Q39:** How many nodes have a degree of one in the branch and bound tree? (That is, they are only connected to one edge). These nodes are called leaf nodes. What is the relationship between the leaf nodes and the remaining feasible region?

**A:** There would be 4 nodes that are only connected to one edge in the tree. Therefore, there are as many leaf nodes as there are regions.

In [ ]:
# Show the next two iterations of the branch and bound algorithm
nodes[3].show()
nodes[4].show()

**Q40:** At the current (dark) node, we added the constraint $x_1 \leq 1$. Why were the fractional solutions $1 < x_1 < 2$ not eliminated for $x_2 <= 3$?

**A:**

In [ ]:
# Show the next three iterations of the branch and bound algorithm
nodes[5].show()
nodes[6].show()
nodes[7].show()

**Q41:** What constraints are enforced at the current (dark) node? Why are there no feasible solutions at this node?

**A:** The constraints being enforced are x1 <= 2, x2 >= 4, and x1 >= 2. The feasible region given by these constraints, in addition to the original ones, gives a region where x2 cannot be greater than 4, yielding no feasible solutions.

In [ ]:
nodes[8].show()

**Q42:** Are we done? If so, what nodes are fathomed and what is the optimal solution? Explain.

**A:** We are done. The nodes fathomed are x1 >= 3, x2 <= 3, x2 ,= 4, and x1 >=2. This yields an optimal integer solution (0,5) with a value of 40.

Let's look at branch and bound visualization for an integer program with 3 decision variables!

In [ ]:
nodes = gilp.bnb_visual(gilp.examples.VARIED_BRANCHING_3D_IP)

In [ ]:
# Look at the first 3 iterations
nodes[0].show()
nodes[1].show()
nodes[2].show()

Let's fast-forward to the final iteration of the branch and bound algorithm.

In [ ]:
nodes[-1].show()

**Q43:** Consider the feasible region that looks like a rectangular box with one corner point at the origin. What node does it correspond to in the tree? What is the optimal solution at that node?

**A:** This corresponds to the node given by the constraints x2 <= 2 x1 <= 1, and x3 <= 2. The optimal solution here is at most 12.

**Q44:** How many branch and bound nodes did we explore? What was the optimal solution? How many branch and bound nodes would we have explored if we knew the value of the optimal solution before starting branch and bound?

**A:** We explored 12 nodes in this problem. If we had known the optimal value beforehand, we would have only had to explore 2 nodes.

## Bonus: Branch and Bound for Knapsack

Consider the following example:

| item | value | weight
|------|-------|-------
|  1   |   2   |   1 
|  2   |   9   |   3 
|  3   |   6   |   2 

The linear program formulation will be:


$$\begin{align*}
\max \quad & 2x_1+9x_2+6x_3\\
\text{s.t.} \quad & 1x_1+3x_2+2x_3 \leq 10 \\
& x_1, x_2, x_3 \geq 0, \quad \text{integer}
\end{align*}$$

In gilp, we can define this lp as follows:

In [ ]:
lp = gilp.LP([[1,3,2]],
             [10],
             [2,9,6])

for fig in gilp.bnb_visual(lp):
    fig.show()